# Setting up the computational model
To simulate the motion of molecules according to the kinetic theory of gases, we need to set up a dynamic n-body simulation with collisions. Note that the objective of this article is to produce a simulation for pedagogical purposes. Therefore, the code is set up to maximize understanding, not execution speed.

## Defining molecular properties
We start by importing all the required modules.

In [1]:
import os
import sys
import time
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

First, we will define a class called Molecule. Objects of this class will store properties such as the masses, positions and velocities of molecules in the simulation. We also define an attribute called color, that may be used to distinguish between different gases and set the color of the points in the animation.

In [ ]:
class Molecule(object):
    #Constructor
    def __init__(self, molecule, position, velocity, mass, color="black"):
        self.molecule = molecule
        self.position = np.array([x_i for x_i in position])
        self.velocity = np.array([v_i for v_i in velocity])
        self.mass = mass
        self.color = color
       
    #Setters for position, velocity, mass and color
    def set_position(self, position):
        self.position = np.array([x_i for x_i in position])
        
    def set_velocity(self,velocity):
        self.velocity = np.array([v_i for v_i in velocity])
        
    def set_color(self, color):
        self.color = color
    
    def set_mass(self, mass):
        self.mass = mass
        
    #Getters for position, velocity, mass and color of the particle
    def get_position(self):
        return self.position
    
    def get_velocity(self):
        return self.velocity
        
    def get_color(self):
        return self.color
        
    def get_mass(self):
        return self.mass

## Adding molecules to a box
Next, we create a Simulation class and define key input parameters, such as the dimensions of the simulation box, that ultimately control the volume of the gas. We also initialize variables for bookkeeping, such as for the wall momentum, that will allow us to calculate pressure.

To add molecules, we first need to initialize their positions and velocities. For both, we need to define functions that create arrays of values based on a distribution input by the user. Note that it does not matter what distributions the initialized positions and velocities follow (as long as there is nothing unphysical, like a molecule outside the box). We shall see that the velocities eventually follow the same distribution.

It is also time to add the molecules to the box. We call the two previously defined functions to generate arrays of initial positions and velocities, create objects of the Molecule class, and assign them to the objects.

Finally, we write a function for general bookkeeping, that is, to make matrices to store the position and velocity vectors as well as their magnitudes. We also make a distance matrix, that stores the distance between every two molecules. This will come in handy to detect collisions.

In [ ]:
class Simulation(object):
    # Constructor
    def __init__(self,name,box_dim,t_step,particle_radius):
        # Set simulation inputs
        self.name = name # Name of the simulation
        self.box_dim = [x for x in box_dim] # Dimensions of the box
        self.t_step = t_step # Timestep
        self.particle_radius = particle_radius # Radius of the particles
        
        # Calculate volume and number of dimensions
        self.V = np.prod(self.box_dim) # Area/Volume of the box
        self.dim = int(len(box_dim)) # Number of dimensions (2D or 3D)
        
        # Initialize paramters
        self.molecules = [] # Create empty list to store objects of class Molecule
        self.n_molecules = 0 # Create variable to store number of molecules
        self.wall_collisions = 0 # Create variable to store number of wall collisions
        self.wall_momentum = 0 # Create variable to store net momentum exchanged with wall

    def _generate_initial_positions(self, n, dist="uniform"):
            #Uniform distribution
            if dist == "uniform":
                _pos = np.random.uniform(low=[0]*self.dim, high=self.box_dim, size=(n,self.dim))
                
            #Store positions in temporary variable    
            self._positions = _pos

    def _generate_initial_velocities(self, n, v_mean, v_std, dist="normal"):
        #Normal distribution with mean v_mean and std v_std
        if dist == "normal":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel=np.random.normal(loc=v_mean, scale=v_std, size=(n,self.dim))
        
        #Uniform distribution with lower bound v_mean and higher bound v_std
        if dist == "uniform":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel = np.random.uniform(low=v_mean, high=v_std, size=(n,self.dim))
        
        #All velocities equal to v_mean
        if dist == "equal":
            self.v_mean = v_mean
            self.v_std = v_std
            _vel = v_mean*np.ones((n,self.dim))
        
        #Randomly switch velocities to negative with probability 0.5
        for i in range(_vel.shape[0]):
            for j in range(_vel.shape[1]):
                if np.random.uniform() > 0.5:
                    _vel[i,j] = -_vel[i,j]
        
        #Store velocities in temporary variable
        self._velocities = _vel
    
    def add_molecules(self, molecule, n, v_mean, v_std, pos_dist="uniform", v_dist="normal", color="black"):
        #Generate initial positions and velocities
        self._generate_initial_positions(n, dist = pos_dist)
        self._generate_initial_velocities(n,v_mean,v_std, dist = v_dist)
        
        #Initialize objects of class Molecule in a list (set mass to 1 as default)
        _add_list=[Molecule(molecule, position = self._positions[i,:], velocity = self._velocities[i,:], \
            color = color, mass = 1) for i in range(n)]
        self.molecules.extend(_add_list)
        self.n_molecules += len(_add_list)
    
    def make_matrices(self):
        #Make empty matrices to store positions, velocities, colors, and masses
        self.positions=np.zeros((self.n_molecules,self.dim))
        self.velocities=np.zeros((self.n_molecules,self.dim))
        self.colors=np.zeros(self.n_molecules,dtype="object")
        self.masses=np.zeros(self.n_molecules)
        
        #Iterate over molecules, get their properties and assign to matrices
        for i,m in enumerate(self.molecules):
            self.positions[i,:]=m.get_position()
            self.velocities[i,:]=m.get_velocity()
            self.colors[i]=m.get_color()
            self.masses[i]=m.get_mass()
        
        #Make vectors with magnitudes of positions and velocities
        self.positions_norm=np.linalg.norm(self.positions,axis=1)
        self.velocities_norm=np.linalg.norm(self.velocities,axis=1)
        
        #Make distance matrix
        self.distance_matrix=np.zeros((self.n_molecules,self.n_molecules))
        for i in range(self.distance_matrix.shape[0]):
            for j in range(self.distance_matrix.shape[1]):
                self.distance_matrix[i,j]=np.linalg.norm(self.positions[i,:]-self.positions[j,:])
                
        #Set diagonal entries (distance with itself) to a high value
        #to prevent them for appearing in the subsequent distance filter
        np.fill_diagonal(self.distance_matrix,1e5)